# It take 1.6 Gb in your google drive

In [1]:
# Mount Google Drive for backup all models
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#@title Download Models  and copy in google drive (If it get banned You can still run it)
!rm -rf /content/RoopFaceSwap
!mkdir /content/RoopFaceSwap
!rm -rf /content/RoopFaceSwap/inswapper
!rm -rf /content/RoopFaceSwap/CodeFormer
%cd /content/RoopFaceSwap
!git clone https://github.com/sczhou/CodeFormer.git

requirements_content = """numpy==1.24.3
opencv-python==4.8.0.74
onnx==1.14.0
insightface==0.7.3
psutil==5.9.5
pillow==10.0.0
onnxruntime-gpu==1.15.1
protobuf==4.23.4
tqdm==4.65.0
addict
future
lmdb
pyyaml
requests
scikit-image
scipy
yapf
lpips
gdown"""

# Specify the file path
file_path = "/content/RoopFaceSwap/requirements.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(requirements_content)

print(f"Requirements file saved at: {file_path}")



code_content = """
# https://github.com/xinntao/BasicSR
# flake8: noqa
from .archs import *
from .data import *
from .losses import *
from .metrics import *
from .models import *
from .ops import *
from .train import *
from .utils import *
# from .version import __gitsha__, __version__
"""

file_path = '/content/RoopFaceSwap/CodeFormer/basicsr/__init__.py'

with open(file_path, 'w') as file:
    file.write(code_content)

print(f"Code has been saved to {file_path}")




from tqdm import tqdm
import urllib
import os
def conditional_download(url,download_file_path):
    if os.path.exists(download_file_path):
      os.remove(download_file_path)
    request = urllib.request.urlopen(url)  # type: ignore[attr-defined]
    total = int(request.headers.get('Content-Length', 0))
    with tqdm(total=total, desc='Downloading', unit='B', unit_scale=True, unit_divisor=1024) as progress:
        urllib.request.urlretrieve(url, download_file_path, reporthook=lambda count, block_size, total_size: progress.update(block_size))  # type: ignore[attr-defined]
import shutil
def move_file(source,dest):
  path=os.path.dirname(dest)
  os.makedirs(path, exist_ok=True)
  shutil.move(source,dest)

import zipfile

def extract_zip(zip_file_path,extract_path):
    # Create the extraction directory if it doesn't exist
    os.makedirs(extract_path, exist_ok=True)

    # Open the ZIP file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all contents to the specified directory
        zip_ref.extractall(extract_path)

    print(f"Contents of '{zip_file_path}' extracted to '{extract_path}'.")



### For face swap
print("inswapper_128.onnx")
conditional_download("https://huggingface.co/countfloyd/deepfake/resolve/main/inswapper_128.onnx","/content/RoopFaceSwap/inswapper_128.onnx") # inswapper_128.onnx
print("buffalo_l.zip")
conditional_download("https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip","/content/RoopFaceSwap/buffalo_l.zip") # buffalo_l.zip link 1
# conditional_download("https://huggingface.co/public-data/insightface/resolve/main/models/buffalo_l.zip","/content/RoopFaceSwap/buffalo_l.zip") # buffalo_l.zip link 2
### For image Upscale
print("codeformer.pth")
conditional_download("https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth","/content/RoopFaceSwap/codeformer.pth")
print("detection_Resnet50_Final.pth")
conditional_download("https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/detection_Resnet50_Final.pth","/content/RoopFaceSwap/detection_Resnet50_Final.pth")
print("parsing_parsenet.pth")
conditional_download("https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/parsing_parsenet.pth","/content/RoopFaceSwap/parsing_parsenet.pth")
print("RealESRGAN_x2plus.pth")
conditional_download("https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/RealESRGAN_x2plus.pth","/content/RoopFaceSwap/RealESRGAN_x2plus.pth")


move_file("/content/RoopFaceSwap/codeformer.pth",'/content/RoopFaceSwap/CodeFormer/weights/CodeFormer/codeformer.pth')
move_file("/content/RoopFaceSwap/detection_Resnet50_Final.pth",'/content/RoopFaceSwap/CodeFormer/weights/facelib/detection_Resnet50_Final.pth')
move_file("/content/RoopFaceSwap/parsing_parsenet.pth",'/content/RoopFaceSwap/CodeFormer/weights/facelib/parsing_parsenet.pth')
move_file("/content/RoopFaceSwap/RealESRGAN_x2plus.pth",'/content/RoopFaceSwap/CodeFormer/weights/realesrgan/RealESRGAN_x2plus.pth')
move_file("/content/RoopFaceSwap/inswapper_128.onnx","/content/RoopFaceSwap/inswapper/inswapper_128.onnx")
extract_zip('/content/RoopFaceSwap/buffalo_l.zip','/content/RoopFaceSwap/inswapper/checkpoints/models/buffalo_l')
os.remove("/content/RoopFaceSwap/buffalo_l.zip")


import os
import shutil
from zipfile import ZipFile

def zip_folder(folder_path, zip_path):
    with ZipFile(zip_path, 'w') as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname=arcname)

from IPython.display import clear_output
clear_output()
folder_to_zip = '/content/RoopFaceSwap'
zip_file_path = '/content/RoopFaceSwap.zip'
zip_folder(folder_to_zip, zip_file_path)
if os.path.exists("/content/gdrive/MyDrive"):
  if not os.path.exists("/content/gdrive/MyDrive/RoopFaceSwap_Backup"):
    os.mkdir("/content/gdrive/MyDrive/RoopFaceSwap_Backup")
  shutil.copy(zip_file_path,"/content/gdrive/MyDrive/RoopFaceSwap_Backup/RoopFaceSwap.zip")